In [1]:
'''
Purpose:Detection of pilots
Changes:number of potential users,train with one SNR
Key words：grant-free，one source blocks, synchronous, m sequence
Date:2019/5/18

Log:1.将用户改为6个，训练次数10000，正确率0.940028
'''
import tensorflow as tf
import numpy as np
import os
from math import*
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3' # 只显示 warning 和 Error

In [ ]:
'''
Purpose:generate pilots mat
length of m sequence : 7
potential users : 6
shape of the matrix : 6*7
'''
def mseq(feedback_index): #导入参数为反馈系数
    m = len(feedback_index)
    global length
    length = 2**m-1 #生成序列的长度
    backdata = tf.Variable(tf.constant(0)) #对应寄存器运算后的值，放在第一个寄存器
    seq = np.zeros((length,),np.int)
    registers = np.ones((m,),np.int)

    for i in range(length):
        seq[i] = registers[m-1]
        # print(seq[i])
        backdata = np.dot(feedback_index,registers)%2
        # print(backdata)
        registers[1:m] = registers[0:(m-1)]
        registers[0] = backdata
        # print(registers)
    # 二进制调制的m序列根序列
    sequence = 1 - 2*seq
    # 定义导频阵列矩阵
    sequence_mat = np.array(np.zeros([length, length]))

    # 通过循环移位产生所有导频序列,构成导频阵列矩阵
    for i in range(length):
        sequence_mat[i, i:length] = sequence[0:length - i]
        sequence_mat[i, 0:i] = sequence[length - i:length]
    # print(sequence_mat)
    sequence_mat = sequence_mat[0:length-1,]
    return sequence_mat

In [3]:
'''
Purpose : 生成含噪声的训练集与验证集,按照MNIST数据集的方式，包含大量不同信噪比噪声的信号，且均匀分布
Parameters : Eb/N0 = 1：10dB
Plan : 根据概率分布对训练集中的数据进行调整
'''
def generater(repetition_base,feedback_index,SNRmax): # 每个Eb/N0的基础重复数量
#     global count
#     count = 0
    image_matrix = np.zeros(7)
    label_matrix = np.zeros(6)
    sequence_mat = mseq(feedback_index)
    Eb_N0_dBs =np.arange(1,SNRmax)
    length_user = 6
    
    repetition_count = 0
    while repetition_count < repetition_base:
        repetition_count = repetition_count + 1
        
        for i in range(2**length_user):
            # i = 2**length-i-1
            # 将值为0~2**length的十进制数转为二进制，之后去掉‘0b’的开头，并将二进制转为列表，此时列表内为字符
            j = list(bin(i).split('b')[1])
            # 将不满长度的列表部分补0`
            add = list(np.zeros([length_user - len(j)], dtype=int))
            add.extend(j)
            # 选择向量：将列表转为数组并将数据类型转为int，至此生成选择向量，0表示该沉默用户，1代表活跃用户
            j = np.array(add).astype(np.int)

            # 通过选择向量选择可能的导频序列进行叠加，即接受端可能收到的信号，共2**length种
            pilots = np.zeros([length])

            for Eb_N0_dB in Eb_N0_dBs:
                variance = 0.5/Eb_N0_dB
                for jj in range(length_user):
                    pilots = pilots + sequence_mat[jj,]*j[jj] + sqrt(variance) * np.random.randn(length)*j[jj]

                image_matrix = np.row_stack((image_matrix, pilots))
                label_matrix = np.row_stack((label_matrix, j))
            
    return image_matrix,label_matrix


In [4]:
'''
Purpose : 生成含噪声的训练集与验证集,按照MNIST数据集的方式，包含大量相同同信噪比噪声的信号
Plan : 根据概率分布对训练集中的数据进行调整
'''
def generater_fixSNR(repetition_base,feedback_index,SNR): 
#     global count
#     count = 0
    image_matrix = np.zeros(7)
    label_matrix = np.zeros(6)
    sequence_mat = mseq(feedback_index)
    
    length_user = 6
    repetition_count = 0
    while repetition_count < repetition_base:
        repetition_count = repetition_count + 1
        
        for i in range(2**length_user):
            # i = 2**length-i-1
            # 将值为0~2**length的十进制数转为二进制，之后去掉‘0b’的开头，并将二进制转为列表，此时列表内为字符
            j = list(bin(i).split('b')[1])
            # 将不满长度的列表部分补0`
            add = list(np.zeros([length_user - len(j)], dtype=int))
            add.extend(j)
            # 选择向量：将列表转为数组并将数据类型转为int，至此生成选择向量，0表示该沉默用户，1代表活跃用户
            j = np.array(add).astype(np.int)

            # 通过选择向量选择可能的导频序列进行叠加，即接受端可能收到的信号，共2**length种
            pilots = np.zeros(length)
            Eb_N0 = SNR
            variance = 0.5/Eb_N0
            for jj in range(length_user):
                pilots = pilots + sequence_mat[jj,]*j[jj] + sqrt(variance) * np.random.randn(length)*j[jj]

            image_matrix = np.row_stack((image_matrix, pilots))
            label_matrix = np.row_stack((label_matrix, j))
            
    return image_matrix,label_matrix

In [5]:
# 提前生成数据集
#生成训练集、验证集，测试集
feedback = np.array([0, 1, 1])  # 反馈序列
train_images, train_labels = generater(30, feedback,10)
test_images, test_labels =generater(10, feedback,10)
validation_images, validation_labels = generater(1, feedback,10)


Instructions for updating:
Colocations handled automatically by placer.


In [16]:
INPUT_NODE = 7  # 输入节点，导频长度是7，所以这里的输入层神经元个数是7
OUTPUT_NODE = 6  # 输出节点，通过每个节点的的值判断是否有活跃用户，当value>0.5时，判断为用户活跃，
                 # 反之不活跃，所以输出层神经元个数为潜在用户数
LAYER1_NODE = 500  # 隐藏层1的神经元个数
LAYER2_NODE = 500  # 隐藏层2的神经元个数

BATCH_SIZE = 100  # 每次batch打包的样本个数

# 模型相关的参数
LEARNING_RATE_BASE = 0.8 # 基础的学习率
LEARNING_RATE_DECAY = 0.99 # 学习率的衰减率
REGULARAZTION_RATE = 0.0001 # 描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS = 10000 # 训练轮数
# MOVING_AVERAGE_DECAY = 0.99 # 滑动平均衰减率

In [17]:
# 全连接神经网络层的计算
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2,weights3, biases3):
    # 不使用滑动平均类
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        layer2 = tf.nn.relu(tf.matmul(layer1, weights2) + biases2)
        return tf.matmul(layer2, weights3) + biases3

    else:
        # 使用滑动平均类
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

In [18]:
def train():

    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    # 生成隐藏层1的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成隐藏层2的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, LAYER2_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[LAYER2_NODE]))
    # 生成输出层的参数。
    weights3 = tf.Variable(tf.truncated_normal([LAYER2_NODE, OUTPUT_NODE], stddev=0.1))
    biases3 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    # 计算不含滑动平均类的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2,weights3,biases3)

    # 定义训练轮数及相关的滑动平均类
    global_step = tf.Variable(0, trainable=False)
    # variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # variables_averages_op = variable_averages.apply(tf.trainable_variables())
    # average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    # print(average_y)
    
    # 计算交叉熵及其平均值，不进行softmax计算
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=y_)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    # 损失函数的计算
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    regularaztion = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularaztion

    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        len(train_labels) / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)

    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step]):
        train_op = tf.no_op(name='train')

    # 计算正确率，先将输出层数组转化为0/1，再将输出层与标签数组中的数进行一对一对比，计算正确率 
    one = tf.ones_like(y)
    zero = tf.zeros_like(y)
    y = tf.where(y < 0.5, x=zero, y=one)

    correct_prediction = tf.equal(y,y_)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    # 初始化会话，并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: validation_images, y_: validation_labels}
        test_feed = {x: test_images, y_: test_labels}

        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                losss = sess.run(loss, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
                print("After %d training step(s), loss is %g " % (i,losss))

            train_feed = {x: train_images[i:BATCH_SIZE+i], y_: train_labels[i:i+BATCH_SIZE]}
            # xs, ys = mnist.train.next_batch(BATCH_SIZE)
            # print(test_feed.shape)
            sess.run(train_op, feed_dict=train_feed)

        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" % (TRAINING_STEPS, test_acc)))


In [19]:
def main():
    # mnist = input_data.read_data_sets("E:/MNIST_data", one_hot=True)

    train()

In [22]:
if __name__=='__main__':
    main()

After 0 training step(s), validation accuracy using average model is 0.463894 
After 0 training step(s), loss is 2.24853 
After 1000 training step(s), validation accuracy using average model is 0.883016 
After 1000 training step(s), loss is 0.870706 
After 2000 training step(s), validation accuracy using average model is 0.923455 
After 2000 training step(s), loss is 0.44395 
After 3000 training step(s), validation accuracy using average model is 0.896592 
After 3000 training step(s), loss is 0.782859 
After 4000 training step(s), validation accuracy using average model is 0.909879 
After 4000 training step(s), loss is 0.883638 
After 5000 training step(s), validation accuracy using average model is 0.80647 
After 5000 training step(s), loss is 4.20908 
After 6000 training step(s), validation accuracy using average model is 0.916233 
After 6000 training step(s), loss is 0.519476 
After 7000 training step(s), validation accuracy using average model is 0.950607 
After 7000 training step(